In [1]:
import pandas as pd

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/tlp/synthetic-tvaes.csv')

In [3]:
df.head()

,demo-genero,demo-fechaEvaluacion,demo-fechaNacimiento,demo-rangoEdad,demo-edad,clin-reservaCognitiva_total,clin-reservaCognitiva_idiomas,clin-reservaCognitiva_ocupacion,clin-reservaCognitiva_escolaridad,clin-ansiedad,...,eval-TLP-Tavec-25_Discriminabilidad-PD,eval-TLP-Tavec-26_Sesgo-PD,eval-TLP-Tavec-27_RI_B_frente_RI_A1-PD,eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD,eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,eval-TLP-Tavec-30_RL_LP_frente_RL_CP-PD,eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,eval-TLP-Tavec-33_Recon_Ac_frente_RCl_LP-PD,ED_2Clases
0,2,2024-02-22T00:00:00,1942-05-08T00:00:00,Rango3,81,12,0,1,3,False,...,84.09,0.14,-20.00,0.00,33.33,37.50,-33.33,-38.46,-7.69,D
1,2,2024-06-26T00:00:00,1948-10-07T00:00:00,Rango3,75,8,0,1,3,True,...,79.55,-0.33,-71.43,12.50,0.00,0.00,0.00,-20.00,-20.00,D
2,2,2024-05-14T00:00:00,1952-05-08T00:00:00,Rango2,72,11,0,0,4,False,...,88.64,-0.60,-14.29,75.00,25.00,-20.00,0.00,-16.67,-16.67,D
3,2,2024-04-10T00:00:00,1964-01-27T00:00:00,Rango1,60,18,2,1,4,False,...,90.91,0.60,60.00,16.67,0.00,50.00,-20.00,-50.00,-37.50,D
4,2,2024-05-14T00:00:00,1959-09-16T00:00:00,Rango1,64,13,0,1,4,False,...,86.36,-0.71,0.00,66.67,0.00,-33.33,0.00,-10.00,-10.00,D


In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((396, 77), (100, 77))

In [5]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/synthetic/tvaes/v3'
    )

In [6]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 2 * 3600,
    auto_stack = True,
    # excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       10.27 GB / 15.94 GB (64.5%)
Disk Space Avail:   82.78 GB / 446.36 GB (18.5%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting an

(_ray_fit pid=20444) [1000]	valid_set's binary_logloss: 0.0912255
(_ray_fit pid=20444) [2000]	valid_set's binary_logloss: 0.0741589
(_ray_fit pid=20444) [3000]	valid_set's binary_logloss: 0.0714719


(_dystack pid=15384) 	0.9744	 = Validation score   (roc_auc)
(_dystack pid=15384) 	2.97s	 = Training   runtime
(_dystack pid=15384) 	0.06s	 = Validation runtime
(_dystack pid=15384) Fitting model: LightGBM_BAG_L1 ... Training model for up to 1184.16s of the 1781.64s of remaining time.
(_dystack pid=15384) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.19%)


(_ray_fit pid=13920) [1000]	valid_set's binary_logloss: 0.146959
(_ray_fit pid=13920) [2000]	valid_set's binary_logloss: 0.118726
(_ray_fit pid=13920) [3000]	valid_set's binary_logloss: 0.10674
(_ray_fit pid=13920) [4000]	valid_set's binary_logloss: 0.10283
(_ray_fit pid=13920) [5000]	valid_set's binary_logloss: 0.102277
(_ray_fit pid=13920) [6000]	valid_set's binary_logloss: 0.102063
(_ray_fit pid=13920) [7000]	valid_set's binary_logloss: 0.10189
(_ray_fit pid=13920) [8000]	valid_set's binary_logloss: 0.101702
(_ray_fit pid=13920) [9000]	valid_set's binary_logloss: 0.101463
(_ray_fit pid=13920) [10000]	valid_set's binary_logloss: 0.10131


(_dystack pid=15384) 	0.9706	 = Validation score   (roc_auc)
(_dystack pid=15384) 	5.56s	 = Training   runtime
(_dystack pid=15384) 	0.04s	 = Validation runtime
(_dystack pid=15384) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 1174.45s of the 1771.93s of remaining time.
(_dystack pid=15384) 	0.9678	 = Validation score   (roc_auc)
(_dystack pid=15384) 	0.61s	 = Training   runtime
(_dystack pid=15384) 	0.09s	 = Validation runtime
(_dystack pid=15384) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 1173.72s of the 1771.21s of remaining time.
(_dystack pid=15384) 	0.9671	 = Validation score   (roc_auc)
(_dystack pid=15384) 	0.55s	 = Training   runtime
(_dystack pid=15384) 	0.09s	 = Validation runtime
(_dystack pid=15384) Fitting model: CatBoost_BAG_L1 ... Training model for up to 1173.05s of the 1770.54s of remaining time.
(_dystack pid=15384) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cp

...

(_dystack pid=15384) 	0.9703	 = Validation score   (roc_auc)
(_dystack pid=15384) 	2.37s	 = Training   runtime
(_dystack pid=15384) 	0.05s	 = Validation runtime
(_dystack pid=15384) Fitting model: NeuralNetFastAI_r143_BAG_L1 ... Training model for up to 193.36s of the 790.84s of remaining time.
(_dystack pid=15384) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.03%)
(_ray_fit pid=3544) No improvement since epoch 0: early stopping
(_dystack pid=15384) 	0.877	 = Validation score   (roc_auc)
(_dystack pid=15384) 	9.97s	 = Training   runtime
(_dystack pid=15384) 	0.14s	 = Validation runtime
(_dystack pid=15384) Fitting model: CatBoost_r70_BAG_L1 ... Training model for up to 178.90s of the 776.38s of remaining time.
(_dystack pid=15384) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.49%)
(_dystack pid=15384) 	0.973	 = Validation sc

(_ray_fit pid=5736) [1000]	valid_set's binary_logloss: 0.296569
(_ray_fit pid=20020) [1000]	valid_set's binary_logloss: 0.269244
(_ray_fit pid=2432) [10000]	valid_set's binary_logloss: 0.0667697 [repeated 21x across cluster]


(_dystack pid=15384) 	0.9742	 = Validation score   (roc_auc)
(_dystack pid=15384) 	7.86s	 = Training   runtime
(_dystack pid=15384) 	0.11s	 = Validation runtime
(_dystack pid=15384) Fitting model: RandomForest_r39_BAG_L1 ... Training model for up to 81.99s of the 679.47s of remaining time.
(_ray_fit pid=15680) No improvement since epoch 18: early stopping [repeated 6x across cluster]
(_dystack pid=15384) 	0.9607	 = Validation score   (roc_auc)
(_dystack pid=15384) 	0.67s	 = Training   runtime
(_dystack pid=15384) 	0.08s	 = Validation runtime
(_dystack pid=15384) Fitting model: CatBoost_r167_BAG_L1 ... Training model for up to 81.21s of the 678.69s of remaining time.
(_dystack pid=15384) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=0.64%)
(_dystack pid=15384) 	0.9704	 = Validation score   (roc_auc)
(_dystack pid=15384) 	46.75s	 = Training   runtime
(_dystack pid=15384) 	0.07s	 = Validation runtime
(_dystack

(_ray_fit pid=8048) [1000]	valid_set's binary_logloss: 0.121387
(_ray_fit pid=3016) [1000]	valid_set's binary_logloss: 0.212606


(_dystack pid=15384) 	0.9766	 = Validation score   (roc_auc)
(_dystack pid=15384) 	2.67s	 = Training   runtime
(_dystack pid=15384) 	0.04s	 = Validation runtime
(_dystack pid=15384) Fitting model: RandomForest_r39_BAG_L2 ... Training model for up to 104.46s of the 104.30s of remaining time.
(_ray_fit pid=8172) No improvement since epoch 4: early stopping [repeated 6x across cluster]
(_dystack pid=15384) 	0.971	 = Validation score   (roc_auc)
(_dystack pid=15384) 	0.67s	 = Training   runtime
(_dystack pid=15384) 	0.07s	 = Validation runtime
(_dystack pid=15384) Fitting model: CatBoost_r167_BAG_L2 ... Training model for up to 103.40s of the 103.23s of remaining time.
(_dystack pid=15384) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=1.00%)
(_dystack pid=15384) 	0.9761	 = Validation score   (roc_auc)
(_dystack pid=15384) 	24.78s	 = Training   runtime
(_dystack pid=15384) 	0.04s	 = Validation runtime
(_dystack 

In [7]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                 WeightedEnsemble_L3   0.983379     roc_auc       1.097649  110.737206                0.000000           0.048567            3      False        220
1                 WeightedEnsemble_L2   0.982409     roc_auc       0.590220   96.988331                0.001000           0.049048            2      False        111
2                   LightGBMXT_BAG_L2   0.982205     roc_auc       0.800679  100.645917                0.031186           2.599107            2      False        112
3                LightGBM_r188_BAG_L2   0.982103     roc_auc       0.797976  100.799530                0.028483           2.752719            2      False        136
4                 LightGBM_r94_BAG_L2   0.981286     roc_auc       0.799735  100.701333                0.030

c:\Users\jgala\uned\tfm\synthetic-data\.conda\Lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetTorch_r79_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiT

In [8]:
auxiliary_metrics = ['accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [9]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9894480519480519
Evaluations on test data:
{
    "roc_auc": 0.9894480519480519,
    "accuracy": 0.93,
    "balanced_accuracy": 0.9301948051948052,
    "mcc": 0.8584755043247572,
    "f1": 0.9369369369369369,
    "precision": 0.9454545454545454,
    "recall": 0.9285714285714286
}


{'roc_auc': 0.9894480519480519,
 'accuracy': 0.93,
 'balanced_accuracy': 0.9301948051948052,
 'mcc': 0.8584755043247572,
 'f1': 0.9369369369369369,
 'precision': 0.9454545454545454,
 'recall': 0.9285714285714286}

In [10]:
predictor.model_best

'WeightedEnsemble_L3_FULL'

In [11]:
predictors = predictor.leaderboard(
    df_test,
    extra_metrics = auxiliary_metrics,
    extra_info=True,
    silent=True
)

In [12]:
predictors[['model', 'roc_auc', 'accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall', 'num_features']]

,model,roc_auc,accuracy,balanced_accuracy,f1,precision,recall,num_features
0,LightGBM_r143_BAG_L1_FULL,0.995942,0.96,0.959416,0.964286,0.964286,0.964286,82
1,LightGBM_BAG_L1_FULL,0.995536,0.97,0.970779,0.972973,0.981818,0.964286,82
2,XGBoost_r31_BAG_L1_FULL,0.994318,0.95,0.950487,0.954955,0.963636,0.946429,82
3,LightGBM_r131_BAG_L1_FULL,0.993101,0.96,0.959416,0.964286,0.964286,0.964286,82
4,XGBoost_r95_BAG_L1_FULL,0.993101,0.93,0.930195,0.936937,0.945455,0.928571,82
...,...,...,...,...,...,...,...,...
435,NeuralNetFastAI_r88_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82
436,NeuralNetFastAI_r160_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82
437,NeuralNetFastAI_r172_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82
438,NeuralNetFastAI_r69_BAG_L1,NaN,NaN,NaN,NaN,NaN,NaN,82


In [13]:
predictors.to_excel(
    '../../data/tlp/predictors-tvaes.xlsx',
    index=False
)

In [14]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['clin-Covid_sintomaSarpullido']
Computing feature importance via permutation shuffling for 75 features using 396 rows with 5 shuffle sets...
	188.79s	= Expected runtime (37.76s per shuffle set)
	19.16s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
eval-TLP-Stroop-color-PD,1.480801e-04,2.599908e-04,0.135895,5,6.834047e-04,-3.872446e-04
eval-TLP-Tavec-1_RI_A1-PD,2.220446e-17,4.965068e-17,0.186950,5,1.244359e-16,-8.002698e-17
eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,2.220446e-17,4.965068e-17,0.186950,5,1.244359e-16,-8.002698e-17
eval-TLP-Tavec-11_RCl_LP-PD,2.220446e-17,4.965068e-17,0.186950,5,1.244359e-16,-8.002698e-17
eval-TLP-Tavec-10_RL_LP-PD,2.220446e-17,4.965068e-17,0.186950,5,1.244359e-16,-8.002698e-17
...,...,...,...,...,...,...
ques-Sus-total-PD,0.000000e+00,0.000000e+00,0.500000,5,0.000000e+00,0.000000e+00
ques-Sus-totalImpar-PD,0.000000e+00,0.000000e+00,0.500000,5,0.000000e+00,0.000000e+00
ques-Sus-totalPar-PD,0.000000e+00,0.000000e+00,0.500000,5,0.000000e+00,0.000000e+00
ques-NavEspacial-resultadosWFQ_DE-PD,0.000000e+00,0.000000e+00,0.500000,5,0.000000e+00,0.000000e+00
